# Plot Cote D'Ivoire Monthly Rainfall Data #2

This will show several plotting examples with comments.

In [ ]:
import glob
import numpy as np
import xarray as xr
import hvplot.xarray
import hvplot.pandas
#import geoviews.feature as gf
import panel as pn
import colorcet as cc

#set the backend and the renderer
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')
br = hv.renderer('bokeh')

In [ ]:
# Set the data directory 
nc_dir = '../data/rainfall/'

## Load Data into Xarray Dataset

Load multiple netCDF-4 files using Xarray. Call compute() to load the data into memory since xr.open_mfdataset() uses Dask to lazily load the files resulting in chunked data. Calling compute() to put all the data into memory.  

In [ ]:
# 1. Load the netCDF-4 file into an Xarray dataset
# 2. Print the Dataset header
dpr_all_ds = xr.open_mfdataset(nc_dir + '*.nc4').compute()
dpr_all_ds

In [ ]:
# # 1. Convert the Xarray Dataset (gridded data) to a Pandas Dataframe (tabular data)
# #    reset_index() causes the Dataframe's first column to be numbers instead of lon/lat/time
# # 2. Print the Dataframe header
# dpr_all_df = dpr_all_ds.to_dataframe().reset_index()
# dpr_all_df

## Monthly Mean Rainfall Map

In [ ]:
# Monthly Mean Rainfall

sf_monthly_mean_rainfall_plot = \
dpr_all_ds.sf_rainfall.groupby('time.month').mean().\
    hvplot.quadmesh(x='lon', y='lat',                                    # Call hvplot's QuadMesh mapping function and set the x and y coordinate variables
                    rasterize=True,                                      # Make it plot faster
                    cmap='rainbow', clim=(0,20),                         # Set the colormap and it's bounds
                    coastline='50m',                                     # Plot a 50 meter resolution coastline
                    frame_width=300,                                     # Plot's width is 300 pixels
                    title='Stratiform Rainfall',
                    ).\
                opts(opts.Feature(line_width=2.0))                       # Additional options - increase the coastline's line width

# Convective Non-shallow Plot
conv_non_sh_monthly_mean_rainfall_plot = \
dpr_all_ds.conv_non_sh_rainfall.groupby('time.month').mean().\
    hvplot.quadmesh(x='lon', y='lat', 
                    rasterize=True, 
                    cmap='rainbow', clim=(0,20),
                    coastline='50m', 
                    frame_width=300,
                    title='Convective Non-shallow Rainfall').\
                opts(opts.Feature(line_width=2.0))

# Convective Shallow Plot
conv_sh_monthly_mean_rainfall_plot = \
dpr_all_ds.conv_sh_rainfall.groupby('time.month').mean().\
    hvplot.quadmesh(x='lon', y='lat', 
                    rasterize=True, 
                    cmap='rainbow', clim=(0,3),
                    coastline='50m', 
                    frame_width=300,
                    title='Convective Shallow Rainfall').\
                opts(opts.Feature(line_width=2.0))

# Set month slider to appear at bottom of the plot
hv.output(widget_location='bottom')

# Plot all 3 in a single row
sf_monthly_mean_rainfall_plot + conv_non_sh_monthly_mean_rainfall_plot + conv_sh_monthly_mean_rainfall_plot

## Monthly Rainfall Anomaly from Annual Climatological Mean

https://xarray.pydata.org/en/v2022.03.0/examples/weather-data.html

https://colorcet.holoviz.org/user_guide/index.html

In [ ]:
# Create Climatological Mean Rainfall 
climo_mean_rainfall_da = dpr_all_ds.all_rainfall.mean('time')
climo_mean_rainfall_da

# Create Monthly Mean Rainfall
monthly_mean_rainfall_da = dpr_all_ds.all_rainfall.groupby('time.month').mean()
monthly_mean_rainfall_da

# Fine the monthly difference from climatological mean
monthly_mean_rainfall_anom_da = monthly_mean_rainfall_da - climo_mean_rainfall_da

# Select colormap, we'll reverse it so green is positive anomaly
cmap = cc.CET_D3[::-1]

# Plot monthly anomaly
monthly_mean_rainfall_anom_da.hvplot.quadmesh(x='lon', y='lat', 
                                              cmap=cmap, clim=(-20,20), clabel='mm/day',
                                              coastline='50m',
                                              xlabel='Longitude', ylabel='Latitude',
                                              frame_width=500,
                                              title='Monthly Rainfall Anomaly').\
                                            opts(opts.Feature(line_width=2.0))